# Basic control structures and variables II

## The Swiss Army Hammer

Yesterday, we covered some central components of Python, such as:

* types, e.g. `int`, `str`

* loops, e.g. `for`, `while`

* modules, e.g. `math`, `numpy`

Today will be a shorter equipment session, followed by more hands on coding. As you should have basic familiarity with coding concepts, I am going to assume object-oriented programming is familiar to you - if not, there is a brief Python-based intro to the key ideas at the bottom ("PS Introduction to objects...")

A couple of basic variables we will use in this module - please re-run this cell now to set them, or later to reset them, if needed.

In [1]:
things_to_do = ['Learn Python', 'Finish PhD', 'Publish research',
                'Accept Nobel prize', 'Inspire a new generation']

my_meetup_dot_com_profile = {
    "first name": "Ignatius",
    "favourite number": 9,
    "favourite programming language": "FORTRAN66",
    3: "is the magic number"
}
my_meetup_dot_com_profile["Interests"] = ["Python2", "Python3", "Scientific Python", "Pottery"]

# Classes

The basic syntax for classes is pretty succinct:

In [2]:
class MyClass(object):
    some_attribute = "default_value"

    def __init__(self, some_attribute, arg2, arg3):
        # Some initialization (over-riding default)
        self.some_attribute = some_attribute

    def some_method(self):
        return self.some_attribute

    def another_method(self, arg1):
        if arg1 == "example":
            output = self.some_method()
            return output

A few points to note: `class` is just a block like any other. Default values can be supplied at its top level, but be careful - often it is better to set defaults in the initializer. Methods are defined as functions, but inside the class block. All of them have a first argument `self`, which is the object itself. Like, say, `this` in C++ (but without dereferencing required).

Python classes don't have a particularly strong approach to private/public access, but starting with an underscore is a conventional approach to indicate a member shouldn't be touched by anything outside the class. We subclass from `object` - additional classes may be added by comma-separation. Subclassing from `object` is actually the default in Python3 if no parentheses (or parent class) are provided - so-called *new-style classes*.

You also see your first magic method - the `__init__` function. This is the constructor, called when the class is created. Here we use it to override a default from the arguments it is passed.

In [3]:
aninstance = MyClass("non-default value", 1, 2)

Here, you see the arguments to the `__init__` function.

In [4]:
aninstance.another_method("example")

'non-default value'

And, one method has called the next and returned the value we asked. We can check the type and dir of this like any other object.

In [5]:
print(type(aninstance))
print(", ".join(dir(aninstance)))

<class '__main__.MyClass'>
__class__, __delattr__, __dict__, __dir__, __doc__, __eq__, __format__, __ge__, __getattribute__, __gt__, __hash__, __init__, __init_subclass__, __le__, __lt__, __module__, __ne__, __new__, __reduce__, __reduce_ex__, __repr__, __setattr__, __sizeof__, __str__, __subclasshook__, __weakref__, another_method, some_attribute, some_method


Note that you can call a override a method in a parent class (for example `a_method` in `MyParentClass`) by defining a method with the same name. If you want to effectively wrap it, you can use the format

```python
class MyClass(MyParentClass):
    ...
    def a_method(self, arg1, arg2):
        val = MyParentClass.mymethod(self, arg1, arg2)
        # Do something to val
        return val
```

Note that you must explicitly pass "`self`" in this case.

### Exercise: University Classes

In [6]:
class Institution:
    location = "(unknown)"

    def __init__(self, location):
        self.location = location

    def __str__(self):
        return "Institution in %s" % self.location


In [7]:
my_institution = Institution("Paris")
str(my_institution)

'Institution in Paris'

In [8]:
class University(Institution):
    location = "(unkonwn)"

    def __init__(self, location):
        self.location = location
    def __str__(self):
      return "University in %s" % self.location

In [9]:
my_institution = University("Belfast")
your_institution = University("Paris")
str(my_institution) == "University in Belfast" and str(your_institution) == "University in Paris"

True

Can you alter the `__str__` magic method so that the final cell prints True?

### Exercise: Going Pottery

By now, this should make sense to you:

In [10]:
delimiter = "  "
delimiter.join(my_meetup_dot_com_profile["Interests"])

'Python2  Python3  Scientific Python  Pottery'

It uses the `join` method on a string (the delimiter) to join the entires of a list together, into one big string.

Below, we subclass `str` and call it `MyString`. It has one method, called `join` to override the usual `join` functionality that strings have - that method only calls the original `join` on `str` so, currently, `MyString` strings behave exactly as any other Python strings.

Exercise: make this overridden `join` method check if the iterable you are joining does _not_ have "Pottery" in it and, if so, make it raise a RuntimeError.

Extension: instead of raising an error, silently replace "Pottery" with "More Python" before passing the iterable to `str.join`

In [11]:
class MyString(str):
    def join(self, iterable):
        if 'Pottery' in iterable:
          print(" RuntimeError You can't join with pottery")
          iterable = [i for i in iterable if i != 'Pottery']
        return str.join(self, iterable)

delimiter = MyString(",")
delimiter.join(my_meetup_dot_com_profile["Interests"])

 RuntimeError You can't join with pottery


'Python2,Python3,Scientific Python'

In [12]:
class MyString(str):
    def join(self, iterable):
        if 'Pottery' in iterable:
          new_iterable=["More Python" if item=="Pottery" else item for item in iterable]
        return str.join(self, new_iterable)

delimiter = MyString(",")
delimiter.join(my_meetup_dot_com_profile["Interests"])

'Python2,Python3,Scientific Python,More Python'

NB: I do not endorse subclassing `str` for real-world applications! From the above you can see that, even though it is a basic type, it is also a class - this is important - but you should also note that supplanting basic types unexpectedly makes code less intuitive and maintainable (again, see Pythonic...)

# Exceptions
## Are something extraordinary

In the words of one of early computing's most idiosyncratic legends, Rear Admiral Dr Grace Murray Hopper, "It is often easier to ask for forgiveness than to ask for permission". A distinguishing feature of Python is that it is built from the ground up with this maxim in mind, known as EAFP. In practice, this means preferring exceptions over tests, so using a `try-except` block (or, elsewhere, `try-catch`, etc.) instead of an `if` statement when checking whether you can perform an action.

Simple examples where you are more likely to use an exception in Python than another language:

* When making a directory

Can you see why this might be useful? If you use an `if` to check for non-existence, there is a potential race condition: by the time you reach the body, it could be made. If you catch the exception, you know it existed exactly when you tried to make it. (Although, the recommended routine, `os.makedirs`, has an optional don't-complain-if-dir-exists argument, which is probably even better)

* When testing file existence

Again with the race condition - this is in fact a security issue, as an attacker can create the file between you checking for existence and opening it for writing. If they create it, of course, then they set the permissions and can see the content regardless of your attempts to block reading.

* Checking type

We talked a lot about duck-typing... the underlying principle is that you never reject input types *as long as they work*. Now imagine you have a routine with an argument `x`, where you want to do one thing if `x` is numeric type and another if it's, say, a string... if you use `if` and check their type, well what if this is some weird subclass of float that your routine has been sent, or the author of this type has carefully implemented all the necessary magic functions to make it quack like a float, but it's a completely unrelated class?

Push the boat out and see if it floats. Try casting to a float and if it doesn't work catch the exception. Then try casting to a string. Everything should cast to a string somehow. Now you have made checking numeric-ness a problem of `float`, which is infinitely more qualified to do this than you are.

* Checking for a dictionary key

Maybe what you think is a dictionary isn't - it's something that, when you request `thingy["something"]` will check whether `"something"` is something it might dynamically add, and, if so, will gladly return the result. If you check first (`if "something" in thingy:`), either the answer is misleading ("no"), or what looks like an innocent if-clause is modifying your dictionary. Moreover, even for an ordinary dictionary, having an if-clause followed by a retrieval hits your dictionary scan twice - try-catch only searches once.

Hopefully this motivates the idea of exceptions before tests - EAFP. Why then have you been repeatedly told not to do this in, as Python calls them, LBYL (Look Before You Leap) languages? The answer is usually that exceptions are horrendously slow and inefficient. In Python this isn't true, by design. They are so fundamental to the language that every loop in fact ends, not with a failing test, but when the iterable throws a specific exception (the `StopIteration` exception).

# Try a try

Here is an example of a piece of code that throws an exception:

In [13]:
something_stupid()
print("Next step")

NameError: name 'something_stupid' is not defined

Python provides lots of useful information about an _exception_ by default - in this case, that `something_stupid` is not defined. However, the default approach interrupts the flow - "Next step" is never printed. Sometimes, we need a more nuanced approach, where we can make decisions about an exception within our code.

The actual syntax is similar to what you will likely have seen in other languages:

In [16]:
try:
    something_stupid()
except:
    print('Doh!')
print('Next step')

Doh!
Next step


Well, the first thing is that we didn't define *something_stupid*. So we get an exception and our extremely generic `except` provides no useful information.

However, "Next step" has been printed - our try-except block has caught the exception, and allowed the flow to continue onto the next command.

## An improvement

Realistically, when catching an exception, you probably want to do something about it. Perhaps print an explanatory error message.

In [17]:
try:
    something_stupid()
except Exception as e:
    print('Doh! You forgot that', e)

Doh! You forgot that name 'something_stupid' is not defined


Definitely better, and note that `print` will cast `e`, the exception, to a string - it is actually a more complex object:

In [18]:
try:
    something_stupid()
except Exception as e:
    print(', '.join(dir(e)))

__cause__, __class__, __context__, __delattr__, __dict__, __dir__, __doc__, __eq__, __format__, __ge__, __getattribute__, __gt__, __hash__, __init__, __init_subclass__, __le__, __lt__, __ne__, __new__, __reduce__, __reduce_ex__, __repr__, __setattr__, __setstate__, __sizeof__, __str__, __subclasshook__, __suppress_context__, __traceback__, args, name, with_traceback


Note in particular that `e` can have arguments when it is thrown - retrieved via `e.args`. Suppose we don't actually want to stop the exception bubbling up, just to some logging or tidy-up on the way through.

In [19]:
try:
    something_stupid()
except Exception as e:
    print(e)

name 'something_stupid' is not defined


In [22]:
try:
    something_stupid()
except Exception as e:
    print('I am *not* cleaning your mess for you, deal with it yourself!')
    raise e

I am *not* cleaning your mess for you, deal with it yourself!


NameError: name 'something_stupid' is not defined

Now we get the same exception we would have got the last time, but the `raise` keyword has kept it moving on past. This is quite useful, as Python exceptions tend to have lots of juicy info we wouldn't want to lose by ending our except block with a boring `print('oh no')` and program exit.

In fact this is how we can throw a brand new exception - we take one of the standard types, e.g. "`RuntimeError`" and `raise` it with an explanatory argument:

In [59]:
raise RuntimeError("Something quite generic went wrong")

RuntimeError: Something quite generic went wrong

# Better yet

This is a very broad error, and a parent class of most of the exceptions we have seen. It doesn't make it easy for calling functions to pin-point what went wrong...

Looking at it from the other end, just as we should be more specific about what we catch, we should be more specific about what we catch. This is a good idea, as, usually when we catch an exception, it is because we expect a particular thing to go wrong. As a case in point:

In [60]:
import os

In [61]:
try:
    os.makedirs(dirname)
except:
    # Great, someone has already created that directory
    # We can carry on!
    pass
# lalalalala...
print("just mucking about with my friend dirname")

just mucking about with my friend dirname


First note `pass`. This is required because every block must have at least one non-comment line - if nothing else is there, we can use `pass` - it is Python's `no-op` if that helps.

Now ask yourself, "did we ever actually define dirname?" No? Then as soon as we use it after our supposed check, we will get an unhandled NameError exception.

We can specify what type of exceptions we catch... dir-already-exists have type `OSError`.

In [62]:
try:
    os.makedirs(dirname)
except OSError:
    print("We had an error creating the directory")
# lalalalala...
print("just mucking about with my friend dirname")

NameError: name 'dirname' is not defined

Now we have stepped out of the way of genuine errors coming through. We can actually handle different exception types in different ways - there are plenty of cases this might be useful, if you want to do something awkward which could fail in five different directions.

### Exercise: Yes, but no, but yes, but no

Can you create a function that will:

 * accept a key and a dictionary as arguments, then
 * throw a KeyError if the key is not in the dictionary,
 * _unless the missing key contains the text "haha" somewhere_ in which case...
 * it should throw a RuntimeError with text "Now you're just having a laugh"

In [87]:
my_dictionary = {
    'hoho': 1,
    'hehe': 2,
    'myhaha': 3
}
# hoho, hehe, myhaha are fine; anything else gives a KeyError, unless it contains the text "haha"

def check_dictionary(key, dct):
  if key not in dct:
        if 'haha' in key:
            raise RuntimeError("Now you're just having a laugh")
        else:
            raise KeyError





check_dictionary('myhaha', my_dictionary) # fine
check_dictionary('hohoho', my_dictionary) # KeyError
check_dictionary('hohohaha', my_dictionary) # RuntimeError

KeyError: 

There are several ways to accomplish this, but focus on try-except.

## More Exceptions

In [88]:
try:
    os.makedirs(dirname)
except OSError:
    # Great, someone has already created that directory
    # We can carry on!
    pass
except NameError:
    print("""
    This is the third code example
    where you haven't defined `dirname`.
    Seriously, catch yourself on.
    """)
# lalalalala...
print("just mucking about with my friend dirname")


    This is the third code example
    where you haven't defined `dirname`.
    Seriously, catch yourself on.
    
just mucking about with my friend dirname


Useful tool just slotted in there: multi-line strings. If you start a string with three quotes, you can keep going on and on until you hit another three.

One final example to show how, for this particular case, you really can be a little more specific.

In [89]:
dirname = "/etc/passwd"
try:
    os.makedirs(dirname)
except OSError:
    # Great, someone has already created that directory
    # We can carry on!
    pass
# lalalalala...
print("just mucking about with my friend", dirname)

just mucking about with my friend /etc/passwd


Emm...we tried to make a directory with the same name as a key system file and are blithely assuming that every failure in doing so is simply because it is an existing directory. Not so good.

In [90]:
import errno

dirname = "/etc/pass"

try:
    os.makedirs(dirname)
except OSError as e:
    if e.errno != errno.EEXIST:
        raise
    # Great, someone has already created that directory
    # We can carry on!
# lalalalala...
print("just mucking about with my friend", dirname)

just mucking about with my friend /etc/pass


This emphasises the fact that subclasses of `Exception` often have additional contextually-relevant properties, which you should use. It also points out that `if` statements still have their place in error-handling!

When `raise` has no argument, it re-raises whichever exception it was that got us into this mess.

# A few important Exceptions
## That you might want to catch

* KeyError - `box_of_tricks["not here"]`
* TypeError - `1 + "banana"`
* IOError - `open('/etc/passwd', 'w')`


Exceptions are often more *pythonic* than pre-checking. [This Python2 doc](https://docs.python.org/2/howto/doanddont.html#exceptions) is a good start on that road. CRUCIAL READING!


# Tuples
## The Ice-List

Tuples are basically frozen lists...

In [91]:
target_coordinates = (56, -5)
print(target_coordinates[0], "N", target_coordinates[1], "E")

56 N -5 E


...but it's fixed, so you can't do this...

In [109]:
# target_coordinates[1] = 38     will show an error as tuple does not support item assignment

If you alter it to set the whole variable (try it), that's fine though - you can alter what `target_coordinates` points to, you just can't alter the tuple content itself.

In [110]:
target_coordinates = (56, -15)
target_coordinates[1]

-15

Lists are constantly changing - length, content, type of content. Really, they aren't much like arrays in compiled languages like C or FORTRAN. They are far too flightly to be used as something like a dictionary index for example. However, supose I have a pair - exactly two elements, one a known `int` and one a known `string`, and it can't change. Well, if you stick two basic types together then why can't you do the same thing with it you can do with basic types? There's no weird changing behaviour going on, so if you freeze a list into something like that, can you use it as a dictionary index?

In short - yes you can!

In [111]:
battleships = {}  # new dict
coordinates = (3,2)
battleships[coordinates] = "HIT"  # a key can be atuple but cannot be a list
battleships

{(3, 2): 'HIT'}

So it doesn't work with a list - try it now with a tuple - swap the square brackets on the *second* line for parens () and re-run

In [112]:
print("At", coordinates, "we have a", battleships[coordinates])

At (3, 2) we have a HIT


**Aside**: A *hashable* type can be used as an index in a dict - tuples are, lists aren't (because they can change, aka are *mutable*)

## Sets

Sets are the final standard collection type - they sound a bit like lists, but share more in common with dictionary keys: they are unordered bags with unique entries. They even get their own operators:

In [113]:
my_set = set([1, 2, 3])
my_set

{1, 2, 3}

In [114]:
your_set = set([1, 3, 5, 3]) # Even though 3 appears twice in the list used to create it, the set has it once
your_set

{1, 3, 5}

In [115]:

we_both_have = my_set & your_set
one_of_us_has = my_set | your_set

we_both_have, one_of_us_has

({1, 3}, {1, 2, 3, 5})

Sets can be created using the same syntax as dictionaries, without the colons: `{1, 3, 4}`

They can be a handy way of deduplicating succinctly: `total_unique_entries = len(set([1, 2, 3, 4, 5, 6, 2, 1, 5]))`

# Lambdas & Generators
## Functional Approaches

These are best understood in practice but I will give a couple of definitions.

__lamdba__ : a very short bit of code, a single action, used as a function

In [116]:
mylambda = (lambda x: x + 18)
mylambda(1)

19

We don't really need the parens on the first line, but they are just to show you that the RHS is entirely a single entity - a lambda. It is assigned to the variable `mylambda`.

In [117]:
mylambda = lambda x: x + 18 # same thing
mylambda(1)

19

Lambdas are handy for passing around very short functions that return a useful value within one single statement. For example, passing a sorting function to a sorting algorithm, or a filtering function that returns a boolean to a data pruning method. In fact, any function, defined with `def` can be assigned to a variable and passed around.

In [118]:
sentence = ["this sentence", "is", "really well sorted"]
key_function = lambda s: s.count(' ')
sorted(sentence, key=key_function)

['is', 'this sentence', 'really well sorted']

In [119]:
dictionary = {'me': 'myself', 'you': 'yourself'}
list(map(lambda pair: pair[0] + pair[1], dictionary.items()))

['memyself', 'youyourself']

### Exercise: Three's a Crowd

Can you write a lambda to filter out all numbers divisible by 3 _or_ more than 3 digits long?

In [120]:
import math

filter_function = lambda n: (n%3)!=0 and len(str(n))<3 # INSERT FUNCTION HERE - SHOULD EVALUATE TO TRUE or FALSE
list(filter(filter_function, [1, 234, 129123, 15, 34, 41991, 14])) # Should give [1, 34, 14]

[1, 34, 14]

### Extension Exercise: Calling a Halt

This folder contains a list of NI Railways Rail Halts in Northern Ireland (`halts.geojson`). Can you complete the snippet below to produce a list:

    ['ADELAIDE', 'BALMORAL', 'BRIDGE END', 'CITY HOSPITAL', 'SYDENHAM']
    
You should use `lambda`, `filter`, `map` and the string methods `startswith` and `endswith`. For reference, a halt is a small station (basically).

In [121]:

import json
with open('halts.geojson', 'r') as halts_f:
    halts_geojson = json.load(halts_f)

#belfast_halts = halts_geojson # replace this line
stations = []
for feature in halts_geojson['features']:
    stations.append(feature['properties']['Station'])

filtered_words = list(filter(lambda x: x.startswith('BELFAST'), stations))
filtered_words = list(map(lambda x: x.replace('BELFAST - ', '').replace(' RAIL HALT', ''), filtered_words))
print(sorted(filtered_words))


['ADELAIDE', 'BALMORAL', 'BRIDGE END', 'CITY HOSPITAL', 'SYDENHAM']


## List Comprehensions

These are __loops inside lists__ ...

This expands on the idea of iterables - entities that churn out successive values, whether strings giving characters, lists of objects, ranges of numbers. So far, all of those have been pre-known before we loop over them. But why? Couldn't we have something that chucks out values as we keep asking?

In [122]:
mylist = [x * 2 for x in range(10)]
mylist

[0, 2, 4, 6, 8, 10, 12, 14, 16, 18]

In [123]:
from math import pi, sin
[sin(pi * x) for x in [0, 0.5, -0.5]]

[0.0, 1.0, -1.0]

This approach can be used in a number of scenarios, not just creating lists (this specific usage is a _list comprehension_ ). Another example is creating dictionaries:

In [124]:
with open('halts.geojson', 'r') as halts_f:
    halts_geojson = json.load(halts_f)

# Note the KEY: VALUE format appearing before the "for"
halts_dictionary = {h['properties']['Station']: h for h in halts_geojson['features']}
halts_dictionary['BELFAST - ADELAIDE RAIL HALT']

{'type': 'Feature',
 'properties': {'Station': 'BELFAST - ADELAIDE RAIL HALT',
  'ID': 13,
  'Type': 'R',
  'Type~Def': '',
  'Easting': 332254,
  'Northing': 371972},
 'geometry': {'type': 'Point',
  'coordinates': [-5.955437428473338, 54.57871560269907]}}

Despite the odd way of creating it, this is indeed a dictionary:

In [125]:
type(halts_dictionary)

dict

### Exercise: End of the Line

Try to replace the ellipsis (...) below to create a dictionary that maps rail halt names to their coordinates.
Hint: start by replacing the ellipsis with `h` to get a print-out of what each `h` has in it.

In [126]:
with open('halts.geojson', 'r') as halts_f:
    halts_geojson = json.load(halts_f)
    for h in halts_geojson['features']:
      print(h)


halts_dictionary = {h['properties']['Station']: h['geometry']['coordinates'] for h in halts_geojson['features']}



{'type': 'Feature', 'properties': {'Station': 'BELFAST - ADELAIDE RAIL HALT', 'ID': 13, 'Type': 'R', 'Type~Def': '', 'Easting': 332254, 'Northing': 371972}, 'geometry': {'type': 'Point', 'coordinates': [-5.955437428473338, 54.57871560269907]}}
{'type': 'Feature', 'properties': {'Station': 'BELFAST - CITY HOSPITAL RAIL HALT', 'ID': 18, 'Type': 'R', 'Type~Def': '', 'Easting': 333163, 'Northing': 373131}, 'geometry': {'type': 'Point', 'coordinates': [-5.940299027187508, 54.58877579877649]}}
{'type': 'Feature', 'properties': {'Station': 'BELFAST - BALMORAL RAIL HALT', 'ID': 14, 'Type': 'R', 'Type~Def': '', 'Easting': 331456, 'Northing': 370851}, 'geometry': {'type': 'Point', 'coordinates': [-5.968279586557941, 54.56885673674175]}}
{'type': 'Feature', 'properties': {'Station': 'BELFAST - BRIDGE END RAIL HALT', 'ID': 16, 'Type': 'R', 'Type~Def': '', 'Easting': 335340, 'Northing': 374643}, 'geometry': {'type': 'Point', 'coordinates': [-5.906524410655043, 54.60187224278238]}}
{'type': 'Feature

NB - do as I say, not as I do: don't use emoji in code... 🙄

### One Answer

In [127]:
result=halts_dictionary if halts_dictionary['BELFAST - ADELAIDE RAIL HALT'] != [-5.955437428473338, 54.57871560269907] else "celebrate, you did it right"
print(result)

celebrate, you did it right


### Selective hearing: conditionals in comprehensions

You can also use `if` in a comprehension:

In [128]:
[x for x in range(10) if x % 2 == 0]

[0, 2, 4, 6, 8]

This is also true for dictionary comprehensions:

In [129]:
{f'key{n}': f'value{n}' for n in range(10) if n % 2 == 0}

{'key0': 'value0',
 'key2': 'value2',
 'key4': 'value4',
 'key6': 'value6',
 'key8': 'value8'}

These can be especially useful when combined with lambdas, or short multiline functions.

In [130]:
import string

def get_station_name(halt):
    name = halt['properties']['Station']
    name = name.replace('BELFAST -', '').replace('RAIL HALT', '').strip()
    return string.capwords(name)

{get_station_name(h): h['geometry']['coordinates'] for h in halts_geojson['features']}

{'Adelaide': [-5.955437428473338, 54.57871560269907],
 'City Hospital': [-5.940299027187508, 54.58877579877649],
 'Balmoral': [-5.968279586557941, 54.56885673674175],
 'Bridge End': [-5.906524410655043, 54.60187224278238],
 'Sydenham': [-5.886439028702296, 54.60553772015698],
 'Dunmurry': [-6.003165502028699, 54.55335836004234],
 'Derriaghy': [-6.018112962940367, 54.54183702611459],
 'Finaghy': [-5.986779356476255, 54.563799283087],
 'Lambeg': [-6.029546191531564, 54.52974232371766],
 'Hilden': [-6.029356397826791, 54.52232479553542],
 'Whiteabbey': [-5.904456204174812, 54.672354744865736],
 'Trooperslane': [-5.848758254028238, 54.710502995359896],
 'Downshire': [-5.789883927272109, 54.72111676355771],
 'Clipperstown': [-5.816523431014697, 54.71743651749125],
 'Whitehead': [-5.709686802660552, 54.75281108416051],
 'Ballycarry': [-5.725569853106984, 54.776620152748315],
 'Magheramorne': [-5.76684603747889, 54.815590324119114],
 'Glynn': [-5.807066008648389, 54.8272213298175],
 'Hollywoo

You might have noticed that you can effectively... map... values from one list into another using a list comprehension. You can also use `if` to... filter... them.

### Exercise: Three in a Row

Replace the filter function and lamba in [Exercise: Three's a Crowd](#Exercise:-Three's-a-Crowd) with a single list comprehension producing the same output.

Can you write a lambda to filter out all numbers divisible by 3 or more than 3 digits long?

In [151]:
# earlier it was without using list comprehension
import math

filter_function = lambda n: (n%3)!=0 and len(str(n))<3 # INSERT FUNCTION HERE - SHOULD EVALUATE TO TRUE or FALSE
list(filter(filter_function, [1, 234, 129123, 15, 34, 41991, 14])) # Should give [1, 34, 14]

[1, 34, 14]

In [154]:
import math
L=[x for x in [1, 234, 129123, 15, 34, 41991, 14] if (x%3)!=0 and len(str(x))<3  ]
L


[1, 34, 14]

### Extension Exercise: More Training

Replace the map, filter and lambda in [Exercise: Calling a Halt](#Extension-Exercise:-Calling-a-Halt) with a dictionary comprehension producing the same output.

In [155]:
# earlier it was without using dict comprehension
import json
with open('halts.geojson', 'r') as halts_f:
    halts_geojson = json.load(halts_f)

#belfast_halts = halts_geojson # replace this line
stations = []
for feature in halts_geojson['features']:
    stations.append(feature['properties']['Station'])

filtered_words = list(filter(lambda x: x.startswith('BELFAST'), stations))
filtered_words = list(map(lambda x: x.replace('BELFAST - ', '').replace(' RAIL HALT', ''), filtered_words))
print(sorted(filtered_words))


['ADELAIDE', 'BALMORAL', 'BRIDGE END', 'CITY HOSPITAL', 'SYDENHAM']


In [157]:

import json
with open('halts.geojson', 'r') as halts_f:
    halts_geojson = json.load(halts_f)

#belfast_halts = halts_geojson # replace this line
filtered_words = sorted({
    feature['properties']['Station']: feature['properties']['Station'].replace('BELFAST - ', '').replace(' RAIL HALT', '')
    for feature in halts_geojson['features']
    if feature['properties']['Station'].startswith('BELFAST')
}.values())

print(filtered_words)

['ADELAIDE', 'BALMORAL', 'BRIDGE END', 'CITY HOSPITAL', 'SYDENHAM']


Lets look at what those same functions could look like if we implemented them using standard if-statements, for loops and functions:

In [131]:
my_list = [1, 234, 129123, 15, 34, 41991, 14]
new_list = []
for n in my_list:
    if (n % 3 > 0) and (n < 1000):
        new_list.append(n)
my_list

[1, 234, 129123, 15, 34, 41991, 14]

In [132]:
all_halts = halts_geojson['features']

belfast_halts = []
for feature in all_halts:
    halt_name = feature['properties']['Station']
    if halt_name.startswith('BELFAST'):
        halt_name = halt_name.replace('BELFAST - ', '').replace(' RAIL HALT', '')
        belfast_halts.append(halt_name)

sorted(belfast_halts)

['ADELAIDE', 'BALMORAL', 'BRIDGE END', 'CITY HOSPITAL', 'SYDENHAM']

Which of the three approaches (filter/map, comprehensions, explicit) do you think would be:

1. easiest-to-understand if you found it in code
2. concise

for each situation? Is it the same for both exercises?

Note that, for list comprehensions and dictionary comprehensions, the entire list or dictionary is generated when you define it. This is not always what you want.

When we open a large file or download, sometimes we will stream it instead of reading the entire entity into memory, so that we can apply a process to each bit without running out of space. For example, when streaming a movie - historically, the entire movie would be downloaded then shown, but video-on-demand will only download enough to show the playing snippet and forget it once it has passed, never holding the entire video into memory.

We can do the same pattern with lists/dictionaries (and iteration in general). These are called _generators_, which provide values on-demand, instead of generating them all up front. An example is `range`...

In [163]:
print([i for i in range(5)])  # you know this already

[0, 1, 2, 3, 4]


In [158]:
my_range = range(10000000)
my_range

range(0, 10000000)

Now we can get a value from it:

In [134]:
my_range[15]

15

That was pretty instantaneous, because range only generates the first 16 entries before returning the value - that's all it needed. If we convert it to a list... (which the `list` function or a list comprehension will do)

In [135]:
my_list = list(range(10000000))
my_list[15]

15

In [136]:
[x for x in range(10000000)][15]

15

In [164]:
[x for x in range(10000000)]

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,


...then it takes much longer, because all the values have to be created before it gets to the next line. 10000000 might seem a big number, but if you had to loop over pixels in an image, or entries in a time-series DB, it would go very quickly.

In [137]:
import sys
sys.getsizeof(my_list), sys.getsizeof(my_range)

(80000056, 48)

Even more so, you can see the relative memory footprints of each variable. You can create your own generators, using functions with the `yield` built-in, or, alternatively, by using an identical syntax to list comprehensions (except with parentheses rather than brackets).

As well as saving memory and time, streams and generators, as a pattern, can let you make condition decisions about how many elements you want to loop through, or even define infinite iterables - for example, for-looping through all the prime numbers until you find one matching a pattern.

### Extension Exercise: Optimal Prime

One to come back to if you have finished other exercises and have some time.

When you call a generator, it will carry on until it hits the first `yield` and return that value. When you request the next, it will carry on _from that yield_ until the next time it hits a yield, and return that value. It keeps returning values as requested until the function exits. Below, we use `primes` to generate a primes - as the function has a `while True` it will never exit, and will keep generating bigger and bigger primes as long as we keep asking for them.

In [138]:
# Fix this, to tell us if n is prime
def is_prime(n):
    if n % 2 == 1:
        return True

# This is a home-brewed generator
def primes():
    n = 2
    while True:
        if is_prime(n):
            yield n
        n = n + 1

In [139]:
# Print all the primes below 1000
for prime in primes():
    if prime > 1000:
        break
    print(prime)

3
5
7
9
11
13
15
17
19
21
23
25
27
29
31
33
35
37
39
41
43
45
47
49
51
53
55
57
59
61
63
65
67
69
71
73
75
77
79
81
83
85
87
89
91
93
95
97
99
101
103
105
107
109
111
113
115
117
119
121
123
125
127
129
131
133
135
137
139
141
143
145
147
149
151
153
155
157
159
161
163
165
167
169
171
173
175
177
179
181
183
185
187
189
191
193
195
197
199
201
203
205
207
209
211
213
215
217
219
221
223
225
227
229
231
233
235
237
239
241
243
245
247
249
251
253
255
257
259
261
263
265
267
269
271
273
275
277
279
281
283
285
287
289
291
293
295
297
299
301
303
305
307
309
311
313
315
317
319
321
323
325
327
329
331
333
335
337
339
341
343
345
347
349
351
353
355
357
359
361
363
365
367
369
371
373
375
377
379
381
383
385
387
389
391
393
395
397
399
401
403
405
407
409
411
413
415
417
419
421
423
425
427
429
431
433
435
437
439
441
443
445
447
449
451
453
455
457
459
461
463
465
467
469
471
473
475
477
479
481
483
485
487
489
491
493
495
497
499
501
503
505
507
509
511
513
515
517
519
521
523
525
527
5

# Decorators

Decorators will be explored more as time permits, but to help you recognise them as we move through, a brief explanation: these are brief annotations directly above `def` (for functions and methods). They tell Python to wrap the function/method with a function-of-a-function. In another phrasing, a decorator takes a function and soups it up. This looks like:

```python
@mydecorator
def afunction():
  ...
```  

In this case `mydecorator` will somehow modify the function's attributes or effects - examples are `@coroutine` for marking a function as async, `@staticmethod` for marking a class's method as static, and `@timer` for timing a function when it is called. You can even make your own - decorators are usually functions with a function as an argument, so you know all the syntax you need. It might look something like this:

In [165]:
def val2string(f):
    def modified_f(val1, val2):
        normal_result = f(val1, val2)
        return str(normal_result)
    return modified_f

The above creates a new function called `modified_f` (note that function definitions can live anywhere, including in a function, just like any other variable definition). It is a two-argument function, which returns the stringified version of the original function, called with the same arguments. For instance:

In [141]:
@val2string
def add(a, b):
    return a + b

@val2string
def mul(a, b):
    return a * b
add(1, 3)

'4'

In [142]:
add(['this'], ['that'])

"['this', 'that']"

In [143]:
mul(6, 2)

'12'

# Wrap-up

This has covered (briefly):

* classes

* exceptions

* lambdas

* comprehensions

# PS: an introduction to objects...

# Objects

## Totally class

If you haven't done object oriented coding before, I am afraid I am not going to give you the magic bullet now. It is an important concept to get to grips with, but for the moment, we will keep it fairly functional. Do take time to look through it later - Python is an excellent language for playing around with it and getting an intuition for what an object is. A number of you will be very familiar with this so please bear with me and the very rough introductory description I will give - for those who are not so familiar, this is worth getting the basics down early.

In real life, everything has properties, and many things have things they can do. In Python these properties are called attributes and the things an object can do are called methods. Together these are called members and, in Python, are accessed by putting a dot after the object and the attribute or method name. Everything else is just like any other variable or function, respectively.

For instance, my dog Freddie has properties - he has a colour, which is black (technically, he is imaginary, but in my head he is definitely black). In Python terms, this would be:

**Attribute**: `freddie.colour = black`

I can tell Freddie to roll-over. When I do so, I am, in some sense, calling Freddie's method:

**Method**: `freddie.roll_over()`

I have another dog, Nitwit. Nitwit can also roll over...

**Method**: `nitwit.roll_over()`

Both dogs can shake hands, but I need to tell them which paw...

**Method**: `freddie.shake_paw('left')`

At this point, you're probably wondering what exactly *is* the set of attributes and methods that my dogs have? This template, showing what attributes and methods a dog of mine can be expected to have, is called a class.

# A simple class

In [144]:
class PhilsDog:
    name = ""
    colour = ""
    def shake_paw(self, side):
        print("My name is", self.name, "and I am shaking my", side, "paw like a good dog")

Here we create a class called `PhilsDog` that all of my dogs *implement* (that is, they are of that type). It indicates that they will have a colour and that they have a method called `shake_paw`. Think of this from the perspective of the dog - the first argument, `self`, is a little Python magic that refers to the dog itself. This lets the dog use it's name and colour (and any other methods) in the `shake_paw` method. The second argument is which paw I told my dog to shake. In response, any dog of mine says "Shaking left/right paw like a good dog". That's quite impressive, but I'd rather they just learned to zip it and actually shake their paws instead.

Now, that's just the template for one of my dogs, so how do I use it?

In [145]:
freddie = PhilsDog()
freddie.name = "Freddie"
freddie.colour = "black"

nitwit = PhilsDog()
nitwit.name = "Nitwit"
nitwit.colour = "brown"

print("Freddie is", freddie.colour, "while Nitwit is", nitwit.colour)

Freddie is black while Nitwit is brown


I call the class like a function. This creates a new PhilsDog object - in computing terminology, I **instantiate** the class, creating a new **instance** of PhilsDog. Freddie is an instance of PhilsDog and so is Nitwit. But PhilsDog is just a template - as you can see, both Freddie and Nitwit have their own colour and name. I can update these just as any variable, and Freddie's doesn't affect Nitwit's, and I can read both back out again.

In [146]:
freddie.shake_paw('left')
nitwit.shake_paw('right')

My name is Freddie and I am shaking my left paw like a good dog
My name is Nitwit and I am shaking my right paw like a good dog


Good dogs. Here we see the `shake_paw` method being called for each dog and with a different `side` parameter. To remind you, the body of the method was:

```python
    def shake_paw(self, side):
        print("My name is", self.name, "and I am shaking my", side, "paw like a good dog")
```

You can see how the `self.name` matches the name I gave each dog on the previous slide.

If that was familiar to you, then that was probably a very unexciting few minutes - if not, it's probably come and gone very quickly. Thankfully, we only need to know that, if we have an object, we can get its attributes and methods by adding a dot and the attribute/method name.

Now why is that important...

# In Python
## Everything is an object (nearly)

In [147]:
"just a normal string".upper()

'JUST A NORMAL STRING'

In [148]:
"Another string".islower()

False

# Two useful tools

In [149]:
type(freddie)

__main__.PhilsDog

"`type`" lets you examine what class an object is. Don't worry about that `__main__` just for the moment.

In [150]:
dir(freddie)[-3:]

['colour', 'name', 'shake_paw']

This is a list of all the members of `freddie`. If he hasn't had any added on the fly (which is possible in Python), this is the same as the members of the `PhilsDog` class. That syntax in the brackets is coming up in a couple of slides, but basically it means, the last three items. Python denotes somewhat magic functions with double-underscores on either side (they are all the previous items I'm hiding away) - there is good reason for these being here, but they aren't essential just now. Try removing the bit in brackets, including the brackets, to see what you get.